In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi) # np.abs : 절대값 계산
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

In [3]:
cap = cv2.VideoCapture(0) # 비디오 캡쳐 객체를 생성할 수 있다. 안의 숫자는 장치 인덱스(어떤 카메라를 사용할 것인가)이다. 1개만 부착되어 있으면 0, 2개 이상이면 첫 웹캠은 0, 두번째 웹캠은 1으로 지정한다.
                          # 0: default camera이다.
counter = 0
posture = None

# squat
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # MIN_DETECTION_CONFIDENCE (최소_탐지_신뢰값), 탐지가 성공한 것으로 간주하는 사람 탐지 모델의 최소 신뢰값은 ([0.0, 1.0])이다. 기본값은 0.5이다.
    # MIN_TRACKING_CONFIDENCE (최소_추적_신뢰값), 탐지가 성공한 것으로 간주하는 사람 탐지 모델의 최소 신뢰값은 ([0.0, 1.0])이다. 추적에 실패하면 다음 이미지 입력에서 사람 감지가 자동으로 호출된다. 추적 신뢰값을 높게 설정하면 솔루션의 견고함이 증가하지만 대기시간도 증가한다. 정적 이미지 모드가 True일 경우 무시된다. 기본값은 0.5이다.
    while cap.isOpened():
        # 카메라 프레임 읽기
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # 이미지 다시쓰기
        
        # 탐지하기
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            # POSE_LANDMARKS (포즈_랜드마크)
            # 포즈 랜드마크의 리스트이다. 각 랜드마크는 다음과 같이 구성된다.
            # x와 y : 랜드마크의 좌표는 각각 이미지의 너비와 높이로 [0.0, 1.0]로 표준화된다.
            # z : 엉덩이 중간 지점의 깊이를 원점으로 하여 랜드마크의 깊이를 나타내며, 랜드마크가 카메라에 가까울수록 값이 작아진다. z의 크기는 x와 거의 비슷한 척도를 사용한다.
            # 가시성 : 이미지에 랜드마크가 보일 가능성(신체가 가려지지 않고 보일 경우) [0.0, 1.0] 가시성의 값이다.
            
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate angle
            left_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_angle = calculate_angle(right_hip, right_knee, right_ankle)
            
            # Visualize
            cv2.putText(image, str(left_angle),
                        tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(right_angle),
                        tuple(np.multiply(right_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
            if left_angle > 155 and right_angle > 155:
                posture = "unknown"
            if left_angle < 125 and right_angle < 125 and posture == "unknown":
                posture = "squat"
                counter += 1
                
        except:
            pass
        
        cv2.rectangle(image, (0,0), (235,73), (245,117,16), -1)
        
        cv2.putText(image, 'REPS', (10,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.putText(image, 'POSTURE', (62,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, posture, (62,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 2, cv2.LINE_AA)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Classify Pose', image) # 프레임 출력

        if cv2.waitKey(10) & 0xFF == ord('q'): # 키보드 입력을 대기하는 함수로 0이면 key 입력이 있을때까지 무한대기한다. 'q'를 누르면 종료한다.
            break

cap.release() # 오픈한 캡쳐 객체를 해제한다.
cv2.destroyAllWindows() # 화면에 나타난 윈도우를 종료한다.
# 계산한 각도에 따라 변화하는 동작마다 카운트